## **Installing Dependencies**

In [ ]:
!pip install tensorflow

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import warnings
warnings.filterwarnings("ignore")

## **Importing Libraries**

In [ ]:
import pandas as pd
import numpy as np
import ast
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

## **Load the data**

Load the data with the hidden states extracted from the LLM

In [ ]:
path = '/content/drive/MyDrive/Ceccarelli_MasterThesis&Internship/Master Thesis/embeddings_saplma/'

In [ ]:
hallu_factalign_1_0_1000 = pd.read_csv(path + 'factalign/embeddings_factalignllama3.2-3B_1_0:1000.csv', index_col=0)
hallu_factalign_4_0_1000 = pd.read_csv(path + 'factalign/embeddings_factalignllama3.2-3B_1_0:1000.csv', index_col=0)
hallu_factalign_8_0_1000 = pd.read_csv(path + 'factalign/embeddings_factalignllama3.2-3B_1_0:1000.csv', index_col = 0)
hallu_factalign_16_0_1000 = pd.read_csv(path + 'factalign/embeddings_factalignllama3.2-3B_1_0:1000.csv', index_col = 0)

hallu_factalign_1_1000_end = pd.read_csv(path + 'factalign/embeddings_factalignllama3.2-3B_1_1000:.csv', index_col=0)
hallu_factalign_4_1000_end = pd.read_csv(path + 'factalign/embeddings_factalignllama3.2-3B_4_1000:.csv', index_col=0)
hallu_factalign_8_1000_end = pd.read_csv(path + 'factalign/embeddings_factalignllama3.2-3B_8_1000:.csv', index_col = 0)
hallu_factalign_16_1000_end = pd.read_csv(path + 'factalign/embeddings_factalignllama3.2-3B_16_1000:.csv', index_col = 0)

hallu_factbench_1 = pd.read_csv(path + 'factbench/embeddings_factbenchllama3.2-3B_1.csv', index_col=0)
hallu_factbench_4 = pd.read_csv(path + 'factbench/embeddings_factbenchllama3.2-3B_4.csv', index_col=0)
hallu_factbench_8 = pd.read_csv(path + 'factbench/embeddings_factbenchllama3.2-3B_8.csv', index_col = 0)
hallu_factbench_16 = pd.read_csv(path + 'factbench/embeddings_factbenchllama3.2-3B_16.csv', index_col = 0)

hallu_felm_1 = pd.read_csv(path + 'felm/embeddings_felmllama3.2-3B_1.csv', index_col=0)
hallu_felm_4 = pd.read_csv(path + 'felm/embeddings_felmllama3.2-3B_4.csv', index_col=0)
hallu_felm_8 = pd.read_csv(path + 'felm/embeddings_felmllama3.2-3B_8.csv', index_col = 0)
hallu_felm_16 = pd.read_csv(path + 'felm/embeddings_felmllama3.2-3B_16.csv', index_col = 0)

In [ ]:
hallu_factalign_1 = pd.concat([hallu_factalign_1_0_1000, hallu_factalign_1_1000_end]).reset_index()
hallu_factalign_4 = pd.concat([hallu_factalign_4_0_1000, hallu_factalign_4_1000_end]).reset_index()
hallu_factalign_8 = pd.concat([hallu_factalign_8_0_1000, hallu_factalign_8_1000_end]).reset_index()
hallu_factalign_16 = pd.concat([hallu_factalign_16_0_1000, hallu_factalign_16_1000_end]).reset_index()

In [ ]:
hallu_factbench_1 = hallu_factbench_1.reset_index()
hallu_factbench_4 = hallu_factbench_4.reset_index()
hallu_factbench_8 = hallu_factbench_8.reset_index()
hallu_factbench_16 = hallu_factbench_16.reset_index()

In [ ]:
hallu_felm_1 = hallu_felm_1.reset_index()
hallu_felm_4 = hallu_felm_4.reset_index()
hallu_felm_8 = hallu_felm_8.reset_index()
hallu_felm_16 = hallu_felm_16.reset_index()

## **Data Preparation**

In [ ]:
#Consider only the prompt, the corresponding response, the factuality label and the embeddings

hallu_factalign_1 = hallu_factalign_1[['prompt', 'completion', 'label', 'embeddings']]
hallu_factalign_4 = hallu_factalign_4[['prompt', 'completion', 'label', 'embeddings']]
hallu_factalign_8 = hallu_factalign_8[['prompt', 'completion', 'label', 'embeddings']]
hallu_factalign_16 = hallu_factalign_16[['prompt', 'completion', 'label', 'embeddings']]

hallu_factbench_1 = hallu_factbench_1[['prompt', 'response', 'response_label', 'embeddings']]
hallu_factbench_4 = hallu_factbench_4[['prompt', 'response', 'response_label', 'embeddings']]
hallu_factbench_8 = hallu_factbench_8[['prompt', 'response', 'response_label', 'embeddings']]
hallu_factbench_16 = hallu_factbench_16[['prompt', 'response', 'response_label', 'embeddings']]

hallu_felm_1 = hallu_felm_1[['prompt', 'response', 'hallucination', 'embeddings']]
hallu_felm_4 = hallu_felm_4[['prompt', 'response', 'hallucination', 'embeddings']]
hallu_felm_8 = hallu_felm_8[['prompt', 'response', 'hallucination', 'embeddings']]
hallu_felm_16 = hallu_felm_16[['prompt', 'response', 'hallucination', 'embeddings']]

In [ ]:
hallu_factbench_1

,prompt,response,response_label,embeddings
0,"Generate a poem with length 10-12 lines, based...",The silence of the night \nIs a peaceful sight...,True,"[-1.7900390625, 0.281494140625, 4.03515625, -1..."
1,Find the average temperature in San Francisco,"As an AI language model, I don't have access t...",False,"[0.47900390625, 0.073974609375, 3.755859375, -..."
2,What is so special about Sir Christopher Wren'...,Sir Christopher Wren's Temple Bar is a histori...,True,"[-0.3896484375, -2.05859375, 3.80078125, -1.65..."
3,"Nicknamed the 'Pilgrims', this football team b...",The team is the New England Patriots.,False,"[0.56298828125, 0.1531982421875, 2.236328125, ..."
4,"Named for a spring, this city in California ha...","The city you are referring to is called ""Los A...",False,"[-0.390869140625, -0.3466796875, -2.650390625,..."
...,...,...,...,...
1986,how many chromosomes are there in a pork chop?,"A pork chop is a cut of meat from a pig, and a...",False,"[1.8623046875, -2.76953125, 3.556640625, -2.20..."
1987,Describe a fashion trend that is currently pop...,"As an AI language model, I do not have a city ...",False,"[1.0068359375, -0.050567626953125, 4.8359375, ..."
1988,Create a speech introducing a friend of yours....,"Ladies and gentlemen, it's my honor to introdu...",True,"[-1.140625, -1.404296875, 3.48046875, -0.50683..."
1989,"Given a current popular movie, generate a funn...",1. Why did the Black Panther refuse to let the...,False,"[-0.896484375, 0.58837890625, -4.0234375, -2.6..."


In [ ]:
# Eliminates any instance without embedding

hallu_factbench_1 = hallu_factbench_1[~hallu_factbench_1['embeddings'].isna()]
hallu_factbench_4 = hallu_factbench_4[~hallu_factbench_4['embeddings'].isna()]
hallu_factbench_8 = hallu_factbench_8[~hallu_factbench_8['embeddings'].isna()]
hallu_factbench_16 = hallu_factbench_16[~hallu_factbench_16['embeddings'].isna()]

In [ ]:
# Evaluates the embedding as list

hallu_factalign_1["embeddings"] = hallu_factalign_1["embeddings"].apply(ast.literal_eval)
hallu_factalign_4["embeddings"] = hallu_factalign_4["embeddings"].apply(ast.literal_eval)
hallu_factalign_8["embeddings"] = hallu_factalign_8["embeddings"].apply(ast.literal_eval)
hallu_factalign_16["embeddings"] = hallu_factalign_16["embeddings"].apply(ast.literal_eval)

hallu_factbench_1["embeddings"] = hallu_factbench_1["embeddings"].apply(ast.literal_eval)
hallu_factbench_4["embeddings"] = hallu_factbench_4["embeddings"].apply(ast.literal_eval)
hallu_factbench_8["embeddings"] = hallu_factbench_8["embeddings"].apply(ast.literal_eval)
hallu_factbench_16["embeddings"] = hallu_factbench_16["embeddings"].apply(ast.literal_eval)

hallu_felm_1["embeddings"] = hallu_felm_1["embeddings"].apply(ast.literal_eval)
hallu_felm_4["embeddings"] = hallu_felm_4["embeddings"].apply(ast.literal_eval)
hallu_felm_8["embeddings"] = hallu_felm_8["embeddings"].apply(ast.literal_eval)
hallu_felm_16["embeddings"] = hallu_felm_16["embeddings"].apply(ast.literal_eval)

In [ ]:
hallu_factalign_1

,prompt,completion,label,embeddings
0,What is the Phaedrus Dialogue?,The Phaedrus Dialogue is a philosophical text ...,False,"[1.2294921875, -3.63671875, 2.365234375, -2.18..."
1,What is the Strait of Gibraltar?,The Strait of Gibraltar is a narrow passage be...,True,"[1.9775390625, -1.6181640625, 2.875, -2.886718..."
2,What is the Pepsi Challenge?,The Pepsi Challenge is a marketing experiment ...,False,"[0.223876953125, -3.845703125, 4.640625, -3.63..."
3,What is the Legal Realism movement?,The Legal Realism movement was a legal philoso...,True,"[1.939453125, -1.958984375, 2.720703125, -1.87..."
4,What occurred during the Tet Offensive?,The Tet Offensive was a surprise military offe...,True,"[0.7998046875, -1.3037109375, 4.47265625, -3.4..."
...,...,...,...,...
2557,What is the Wall Advisory Opinion by the Inter...,The Wall Advisory Opinion by the International...,False,"[2.03515625, -0.0665283203125, 0.94140625, -1...."
2558,What is the Blue Brain Project?,The Blue Brain Project is a research initiativ...,False,"[-0.4931640625, -3.30078125, 1.4609375, -1.696..."
2559,Who is Jean Piaget?,Jean Piaget (1896-1980) was a Swiss psychologi...,True,"[-2.890625, -1.5771484375, 1.291015625, -4.125..."
2560,What occurred with the Cuban Missile Crisis?,The Cuban Missile Crisis was a period of heigh...,False,"[0.5625, -2.291015625, 4.1875, -2.48828125, -0..."


In [ ]:
hallu_factbench_1

,prompt,response,response_label,embeddings
0,"Generate a poem with length 10-12 lines, based...",The silence of the night \nIs a peaceful sight...,True,"[-1.7900390625, 0.281494140625, 4.03515625, -1..."
1,Find the average temperature in San Francisco,"As an AI language model, I don't have access t...",False,"[0.47900390625, 0.073974609375, 3.755859375, -..."
2,What is so special about Sir Christopher Wren'...,Sir Christopher Wren's Temple Bar is a histori...,True,"[-0.3896484375, -2.05859375, 3.80078125, -1.65..."
3,"Nicknamed the 'Pilgrims', this football team b...",The team is the New England Patriots.,False,"[0.56298828125, 0.1531982421875, 2.236328125, ..."
4,"Named for a spring, this city in California ha...","The city you are referring to is called ""Los A...",False,"[-0.390869140625, -0.3466796875, -2.650390625,..."
...,...,...,...,...
1986,how many chromosomes are there in a pork chop?,"A pork chop is a cut of meat from a pig, and a...",False,"[1.8623046875, -2.76953125, 3.556640625, -2.20..."
1987,Describe a fashion trend that is currently pop...,"As an AI language model, I do not have a city ...",False,"[1.0068359375, -0.050567626953125, 4.8359375, ..."
1988,Create a speech introducing a friend of yours....,"Ladies and gentlemen, it's my honor to introdu...",True,"[-1.140625, -1.404296875, 3.48046875, -0.50683..."
1989,"Given a current popular movie, generate a funn...",1. Why did the Black Panther refuse to let the...,False,"[-0.896484375, 0.58837890625, -4.0234375, -2.6..."


In [ ]:
hallu_felm_1

,prompt,response,hallucination,embeddings
0,"Nicknamed the 'Pilgrims', this football team b...",The team is the New England Patriots.,False,"[0.56298828125, 0.1531982421875, 2.236328125, ..."
1,Where did fortune cookies originate,"""Fortune cookies originated in Japan in the 19...",False,"[-0.146728515625, -1.8935546875, -1.4091796875..."
2,Mrs. Tatiana owns a grocery store that sells d...,"After one year, the price of carrots would be ...",False,"[1.669921875, 0.461669921875, 4.1953125, -1.42..."
3,What is the average height of female president...,There have been no female presidents of the Un...,True,"[-0.326904296875, -1.6904296875, 1.6044921875,..."
4,Determine the modulo 4 remainder of the follow...,"We can pair the terms $1$ and $3$, $2$ and $4$...",False,"[2.45703125, -0.5263671875, -0.85791015625, -4..."
...,...,...,...,...
557,Simplify: $\frac{18!}{16!}$,We can simplify this expression by canceling o...,True,"[1.095703125, 2.72265625, 1.1923828125, -3.847..."
558,Which is bigger 6 times 3 plus 2 or 6 plus 3 t...,"To solve this problem, we need to follow the o...",True,"[1.5146484375, 0.7568359375, 3.091796875, -1.9..."
559,If Mark weighs 150 pounds and Susan weighs 20 ...,"Susan weighs 20 pounds less than Mark, so she ...",True,"[-0.254638671875, -1.1416015625, 4.12890625, -..."
560,Who was the CEO of Twitter?,The CEO of Twitter at the time of writing this...,False,"[0.83984375, -1.6240234375, 2.708984375, -1.44..."


In [ ]:
# Renames the columns in the same way for consistency

hallu_factalign_1 = hallu_factalign_1.rename(columns={'completion': 'response'})
hallu_factalign_4 = hallu_factalign_4.rename(columns={'completion': 'response'})
hallu_factalign_8 = hallu_factalign_8.rename(columns={'completion': 'response'})
hallu_factalign_16 = hallu_factalign_16.rename(columns={'completion': 'response'})

hallu_factbench_1 = hallu_factbench_1.rename(columns={'response_label': 'label'})
hallu_factbench_4 = hallu_factbench_4.rename(columns={'response_label': 'label'})
hallu_factbench_8 = hallu_factbench_8.rename(columns={'response_label': 'label'})
hallu_factbench_16 = hallu_factbench_16.rename(columns={'response_label': 'label'})

hallu_felm_1 = hallu_felm_1.rename(columns={'hallucination': 'label'})
hallu_felm_4 = hallu_felm_4.rename(columns={'hallucination': 'label'})
hallu_felm_8 = hallu_felm_8.rename(columns={'hallucination': 'label'})
hallu_felm_16 = hallu_felm_16.rename(columns={'hallucination': 'label'})

In [ ]:
# Removes any instance without factuality label

hallu_factbench_1 = hallu_factbench_1[~hallu_factbench_1['label'].isna()]
hallu_factbench_4 = hallu_factbench_4[~hallu_factbench_4['label'].isna()]
hallu_factbench_8 = hallu_factbench_8[~hallu_factbench_8['label'].isna()]
hallu_factbench_16 = hallu_factbench_16[~hallu_factbench_16['label'].isna()]

In [ ]:
# Convert True and False in 1/0

hallu_factbench_1['label'] = hallu_factbench_1['label'].astype(int)
hallu_factbench_4['label'] = hallu_factbench_4['label'].astype(int)
hallu_factbench_8['label'] = hallu_factbench_8['label'].astype(int)
hallu_factbench_16['label'] = hallu_factbench_16['label'].astype(int)

hallu_felm_1['label'] = hallu_felm_1['label'].astype(int)
hallu_felm_4['label'] = hallu_felm_4['label'].astype(int)
hallu_felm_8['label'] = hallu_felm_8['label'].astype(int)
hallu_felm_16['label'] = hallu_felm_16['label'].astype(int)

hallu_factalign_1['label'] = hallu_factalign_1['label'].astype(int)
hallu_factalign_4['label'] = hallu_factalign_4['label'].astype(int)
hallu_factalign_8['label'] = hallu_factalign_8['label'].astype(int)
hallu_factalign_16['label'] = hallu_factalign_16['label'].astype(int)

## **Training**

In [ ]:
# Split the data in train and test, using a random seed for consistency

random.seed(777)

felm_train_1, felm_test_1 = train_test_split(hallu_felm_1, test_size=0.2, random_state=42)
felm_train_4, felm_test_4 = train_test_split(hallu_felm_4, test_size=0.2, random_state=42)
felm_train_8, felm_test_8 = train_test_split(hallu_felm_8, test_size=0.2, random_state=42)
felm_train_16, felm_test_16 = train_test_split(hallu_felm_16, test_size=0.2, random_state=42)

factalign_train_1, factalign_test_1 = train_test_split(hallu_factalign_1, test_size=0.2, random_state=42)
factalign_train_4, factalign_test_4 = train_test_split(hallu_factalign_4, test_size=0.2, random_state=42)
factalign_train_8, factalign_test_8 = train_test_split(hallu_factalign_8, test_size=0.2, random_state=42)
factalign_train_16, factalign_test_16 = train_test_split(hallu_factalign_16, test_size=0.2, random_state=42)

factbench_train_1, factbench_test_1 = train_test_split(hallu_factbench_1, test_size=0.2, random_state=42)
factbench_train_4, factbench_test_4 = train_test_split(hallu_factbench_4, test_size=0.2, random_state=42)
factbench_train_8, factbench_test_8 = train_test_split(hallu_factbench_8, test_size=0.2, random_state=42)
factbench_train_16, factbench_test_16 = train_test_split(hallu_factbench_16, test_size=0.2, random_state=42)

In [ ]:
# Transform the embedding in numpy array

## FELM

X_train_felm_1 = np.array(felm_train_1['embeddings'].tolist())
X_test_felm_1 = np.array(felm_test_1['embeddings'].tolist())

X_train_felm_4 = np.array(felm_train_4['embeddings'].tolist())
X_test_felm_4 = np.array(felm_test_4['embeddings'].tolist())

X_train_felm_8 = np.array(felm_train_8['embeddings'].tolist())
X_test_felm_8 = np.array(felm_test_8['embeddings'].tolist())

X_train_felm_16 = np.array(felm_train_16['embeddings'].tolist())
X_test_felm_16 = np.array(felm_test_16['embeddings'].tolist())

X_train_felm_all = np.concatenate((X_train_felm_1, X_train_felm_4, X_train_felm_8, X_train_felm_16), axis=1)
X_test_felm_all = np.concatenate((X_test_felm_1, X_test_felm_4, X_test_felm_8, X_test_felm_16), axis=1)

## FACTALIGN

X_train_factalign_1 = np.array(factalign_train_1['embeddings'].tolist())
X_test_factalign_1 = np.array(factalign_test_1['embeddings'].tolist())

X_train_factalign_4 = np.array(factalign_train_4['embeddings'].tolist())
X_test_factalign_4 = np.array(factalign_test_4['embeddings'].tolist())

X_train_factalign_8 = np.array(factalign_train_8['embeddings'].tolist())
X_test_factalign_8 = np.array(factalign_test_8['embeddings'].tolist())

X_train_factalign_16 = np.array(factalign_train_16['embeddings'].tolist())
X_test_factalign_16 = np.array(factalign_test_16['embeddings'].tolist())

X_train_factalign_all = np.concatenate((X_train_factalign_1, X_train_factalign_4, X_train_factalign_8, X_train_factalign_16), axis=1)
X_test_factalign_all = np.concatenate((X_test_factalign_1, X_test_factalign_4, X_test_factalign_8, X_test_factalign_16), axis=1)

## FACTBENCH

X_train_factbench_1 = np.array(factbench_train_1['embeddings'].tolist())
X_test_factbench_1 = np.array(factbench_test_1['embeddings'].tolist())

X_train_factbench_4 = np.array(factbench_train_4['embeddings'].tolist())
X_test_factbench_4 = np.array(factbench_test_4['embeddings'].tolist())

X_train_factbench_8 = np.array(factbench_train_8['embeddings'].tolist())
X_test_factbench_8 = np.array(factbench_test_8['embeddings'].tolist())

X_train_factbench_16 = np.array(factbench_train_16['embeddings'].tolist())
X_test_factbench_16 = np.array(factbench_test_16['embeddings'].tolist())

X_train_factbench_all = np.concatenate((X_train_factbench_1, X_train_factbench_4, X_train_factbench_8, X_train_factbench_16), axis=1)
X_test_factbench_all = np.concatenate((X_test_factbench_1, X_test_factbench_4, X_test_factbench_8, X_test_factbench_16), axis=1)

In [ ]:
print(X_train_factalign_1.shape)
print(X_test_factalign_1.shape)
print(X_train_factbench_1.shape)
print(X_test_factbench_1.shape)
print(X_train_felm_1.shape)
print(X_test_felm_1.shape)

(2049, 3072)
(513, 3072)
(1592, 3072)
(398, 3072)
(449, 3072)
(113, 3072)


In [ ]:
# Transform the list of labels in numpy array

## FELM

y_train_felm_1 = np.array(felm_train_1['label'].tolist())
y_test_felm_1 = np.array(felm_test_1['label'].tolist())

y_train_felm_4 = np.array(felm_train_4['label'].tolist())
y_test_felm_4 = np.array(felm_test_4['label'].tolist())

y_train_felm_8 = np.array(felm_train_8['label'].tolist())
y_test_felm_8 = np.array(felm_test_8['label'].tolist())

y_train_felm_16 = np.array(felm_train_16['label'].tolist())
y_test_felm_16 = np.array(felm_test_16['label'].tolist())

## FACTALIGN

y_train_factalign_1 = np.array(factalign_train_1['label'].tolist())
y_test_factalign_1 = np.array(factalign_test_1['label'].tolist())

y_train_factalign_4 = np.array(factalign_train_4['label'].tolist())
y_test_factalign_4 = np.array(factalign_test_4['label'].tolist())

y_train_factalign_8 = np.array(factalign_train_8['label'].tolist())
y_test_factalign_8 = np.array(factalign_test_8['label'].tolist())

y_train_factalign_16 = np.array(factalign_train_16['label'].tolist())
y_test_factalign_16 = np.array(factalign_test_16['label'].tolist())

## FACTBENCH

y_train_factbench_1 = np.array(factbench_train_1['label'].tolist())
y_test_factbench_1 = np.array(factbench_test_1['label'].tolist())

y_train_factbench_4 = np.array(factbench_train_4['label'].tolist())
y_test_factbench_4 = np.array(factbench_test_4['label'].tolist())

y_train_factbench_8 = np.array(factbench_train_8['label'].tolist())
y_test_factbench_8 = np.array(factbench_test_8['label'].tolist())

y_train_factbench_16 = np.array(factbench_train_16['label'].tolist())
y_test_factbench_16 = np.array(factbench_test_16['label'].tolist())

## **SVM**

In [ ]:
# Defining all the SVM classifiers for the classification of the hidden states

clf_felm_1 = SVC()
clf_factalign_1 = SVC()
clf_factbench_1 = SVC()

clf_felm_4 = SVC()
clf_factalign_4 = SVC()
clf_factbench_4 = SVC()

clf_felm_8 = SVC()
clf_factalign_8 = SVC()
clf_factbench_8 = SVC()

clf_felm_16 = SVC()
clf_factalign_16 = SVC()
clf_factbench_16 = SVC()

clf_felm_all = SVC()
clf_factalign_all = SVC()
clf_factbench_all = SVC()

In [ ]:
# Training of the SVM classifiers

clf_felm_1.fit(X_train_felm_1, y_train_felm_1)
clf_factalign_1.fit(X_train_factalign_1, y_train_factalign_1)
clf_factbench_1.fit(X_train_factbench_1, y_train_factbench_1)

clf_felm_4.fit(X_train_felm_4, y_train_felm_4)
clf_factalign_4.fit(X_train_factalign_4, y_train_factalign_4)
clf_factbench_4.fit(X_train_factbench_4, y_train_factbench_4)

clf_felm_8.fit(X_train_felm_8, y_train_felm_8)
clf_factalign_8.fit(X_train_factalign_8, y_train_factalign_8)
clf_factbench_8.fit(X_train_factbench_8, y_train_factbench_8)

clf_felm_16.fit(X_train_felm_16, y_train_felm_16)
clf_factalign_16.fit(X_train_factalign_16, y_train_factalign_16)
clf_factbench_16.fit(X_train_factbench_16, y_train_factbench_16)

clf_felm_all.fit(X_train_felm_all, y_train_felm_1)
clf_factalign_all.fit(X_train_factalign_all, y_train_factalign_1)
clf_factbench_all.fit(X_train_factbench_all, y_train_factbench_1)

SVC()

In [ ]:
# Computes the results using the test set against the ground truths

result_svm_felm_1 = accuracy_score(y_test_felm_1, clf_felm_1.predict(X_test_felm_1))
result_svm_felm_4 = accuracy_score(y_test_felm_4, clf_felm_4.predict(X_test_felm_4))
result_svm_felm_8 = accuracy_score(y_test_felm_8, clf_felm_8.predict(X_test_felm_8))
result_svm_felm_16 = accuracy_score(y_test_felm_16, clf_felm_16.predict(X_test_felm_16))

result_svm_factalign_1 = accuracy_score(y_test_factalign_1, clf_factalign_1.predict(X_test_factalign_1))
result_svm_factalign_4 = accuracy_score(y_test_factalign_4, clf_factalign_4.predict(X_test_factalign_4))
result_svm_factalign_8 = accuracy_score(y_test_factalign_8, clf_factalign_8.predict(X_test_factalign_8))
result_svm_factalign_16 = accuracy_score(y_test_factalign_16, clf_factalign_16.predict(X_test_factalign_16))

result_svm_factbench_1 = accuracy_score(y_test_factbench_1, clf_factbench_1.predict(X_test_factbench_1))
result_svm_factbench_4 = accuracy_score(y_test_factbench_4, clf_factbench_4.predict(X_test_factbench_4))
result_svm_factbench_8 = accuracy_score(y_test_factbench_8, clf_factbench_8.predict(X_test_factbench_8))
result_svm_factbench_16 = accuracy_score(y_test_factbench_16, clf_factbench_16.predict(X_test_factbench_16))

result_svm_felm_all = accuracy_score(y_test_felm_1, clf_felm_all.predict(X_test_felm_all))
result_svm_factalign_all = accuracy_score(y_test_factalign_1, clf_factalign_all.predict(X_test_factalign_all))
result_svm_factbench_all = accuracy_score(y_test_factbench_1, clf_factbench_all.predict(X_test_factbench_all))

## **Logistic Regression**

In [ ]:
# Defining all the Logistic Regression classifiers for the classification of the hidden states

lr_felm_1 = LogisticRegression()
lr_factalign_1 = LogisticRegression()
lr_factbench_1 = LogisticRegression()

lr_felm_4 = LogisticRegression()
lr_factalign_4 = LogisticRegression()
lr_factbench_4 = LogisticRegression()

lr_felm_8 = LogisticRegression()
lr_factalign_8 = LogisticRegression()
lr_factbench_8 = LogisticRegression()

lr_felm_16 = LogisticRegression()
lr_factalign_16 = LogisticRegression()
lr_factbench_16 = LogisticRegression()

lr_felm_all = LogisticRegression()
lr_factalign_all = LogisticRegression()
lr_factbench_all = LogisticRegression()

In [ ]:
# Training of the logistic regression classifiers

lr_felm_1.fit(X_train_felm_1, y_train_felm_1)
lr_factalign_1.fit(X_train_factalign_1, y_train_factalign_1)
lr_factbench_1.fit(X_train_factbench_1, y_train_factbench_1)

lr_felm_4.fit(X_train_felm_4, y_train_felm_4)
lr_factalign_4.fit(X_train_factalign_4, y_train_factalign_4)
lr_factbench_4.fit(X_train_factbench_4, y_train_factbench_4)

lr_felm_8.fit(X_train_felm_8, y_train_felm_8)
lr_factalign_8.fit(X_train_factalign_8, y_train_factalign_8)
lr_factbench_8.fit(X_train_factbench_8, y_train_factbench_8)

lr_felm_16.fit(X_train_felm_16, y_train_felm_16)
lr_factalign_16.fit(X_train_factalign_16, y_train_factalign_16)
lr_factbench_16.fit(X_train_factbench_16, y_train_factbench_16)

lr_felm_all.fit(X_train_felm_all, y_train_felm_1)
lr_factalign_all.fit(X_train_factalign_all, y_train_factalign_1)
lr_factbench_all.fit(X_train_factbench_all, y_train_factbench_1)

LogisticRegression()

In [ ]:
# Computes the results using the test set against the ground truths

result_lr_felm_1 = accuracy_score(y_test_felm_1, lr_felm_1.predict(X_test_felm_1))
result_lr_felm_4 = accuracy_score(y_test_felm_4, lr_felm_4.predict(X_test_felm_4))
result_lr_felm_8 = accuracy_score(y_test_felm_8, lr_felm_8.predict(X_test_felm_8))
result_lr_felm_16 = accuracy_score(y_test_felm_16, lr_felm_16.predict(X_test_felm_16))

result_lr_factalign_1 = accuracy_score(y_test_factalign_1, lr_factalign_1.predict(X_test_factalign_1))
result_lr_factalign_4 = accuracy_score(y_test_factalign_4, lr_factalign_4.predict(X_test_factalign_4))
result_lr_factalign_8 = accuracy_score(y_test_factalign_8, lr_factalign_8.predict(X_test_factalign_8))
result_lr_factalign_16 = accuracy_score(y_test_factalign_16, lr_factalign_16.predict(X_test_factalign_16))

result_lr_factbench_1 = accuracy_score(y_test_factbench_1, lr_factbench_1.predict(X_test_factbench_1))
result_lr_factbench_4 = accuracy_score(y_test_factbench_4, lr_factbench_4.predict(X_test_factbench_4))
result_lr_factbench_8 = accuracy_score(y_test_factbench_8, lr_factbench_8.predict(X_test_factbench_8))
result_lr_factbench_16 = accuracy_score(y_test_factbench_16, lr_factbench_16.predict(X_test_factbench_16))

result_lr_felm_all = accuracy_score(y_test_felm_1, lr_felm_all.predict(X_test_felm_all))
result_lr_factalign_all = accuracy_score(y_test_factalign_1, lr_factalign_all.predict(X_test_factalign_all))
result_lr_factbench_all = accuracy_score(y_test_factbench_1, lr_factbench_all.predict(X_test_factbench_all))

## **Dataframes with metrics**

In [ ]:
metrics_felm_svm = {
    'classifier' : ['svm', 'svm', 'svm', 'svm', 'svm'],
    'layer' : [-1, -4, -8, -16, 'all'],
    'accuracy' : [result_svm_felm_1, result_svm_felm_4, result_svm_felm_8, result_svm_felm_16, result_svm_felm_all]
}

metrics_felm_lr = {
    'classifier' : ['lr', 'lr', 'lr', 'lr', 'lr'],
    'layer' : [-1, -4, -8, -16, 'all'],
    'accuracy' : [result_lr_felm_1, result_lr_felm_4, result_lr_felm_8, result_lr_felm_16, result_lr_felm_all]
}

metrics_factalign_svm = {
    'classifier' : ['svm', 'svm', 'svm', 'svm', 'svm'],
    'layer' : [-1, -4, -8, -16, 'all'],
    'accuracy' : [result_svm_factalign_1, result_svm_factalign_4, result_svm_factalign_8, result_svm_factalign_16, result_svm_factalign_all]
}

metrics_factalign_lr = {
    'classifier' : ['lr', 'lr', 'lr', 'lr', 'lr'],
    'layer' : [-1, -4, -8, -16, 'all'],
    'accuracy' : [result_lr_factalign_1, result_lr_factalign_4, result_lr_factalign_8, result_lr_factalign_16, result_lr_factalign_all]
}

metrics_factbench_svm = {
    'classifier' : ['svm', 'svm', 'svm', 'svm', 'svm'],
    'layer' : [-1, -4, -8, -16, 'all'],
    'accuracy' : [result_svm_factbench_1, result_svm_factbench_4, result_svm_factbench_8, result_svm_factbench_16, result_svm_factbench_all]
}

metrics_factbench_lr = {
    'classifier' : ['lr', 'lr', 'lr', 'lr', 'lr'],
    'layer' : [-1, -4, -8, -16, 'all'],
    'accuracy' : [result_lr_factbench_1, result_lr_factbench_4, result_lr_factbench_8, result_lr_factbench_16, result_lr_factbench_all]
}

In [ ]:
df_metrics_felm_svm = pd.DataFrame(metrics_felm_svm)
df_metrics_felm_lr = pd.DataFrame(metrics_felm_lr)

df_metrics_factalign_svm = pd.DataFrame(metrics_factalign_svm)
df_metrics_factalign_lr = pd.DataFrame(metrics_factalign_lr)

df_metrics_factbench_svm = pd.DataFrame(metrics_factbench_svm)
df_metrics_factbench_lr = pd.DataFrame(metrics_factbench_lr)

In [ ]:
df_metrics_felm = pd.concat([df_metrics_felm_svm, df_metrics_felm_lr])
df_metrics_factalign = pd.concat([df_metrics_factalign_svm, df_metrics_factalign_lr])
df_metrics_factbench = pd.concat([df_metrics_factbench_svm, df_metrics_factbench_lr])

In [ ]:
df_metrics_factalign

,classifier,layer,accuracy
0,svm,-1,0.730994
1,svm,-4,0.707602
2,svm,-8,0.709552
3,svm,-16,0.668616
4,svm,all,0.715400
0,lr,-1,0.674464
1,lr,-4,0.680312
2,lr,-8,0.699805
3,lr,-16,0.688109
4,lr,all,0.690058


## **Neural Network - 3 layers (SAPLMA)**

In [ ]:
## Function that define the deep neural networks with three layers

def define_model_3layer(input_dim = X_train_felm_1.shape[1]):

  model = Sequential()
  model.add(Dense(256, activation='relu', input_dim=input_dim))
  model.add(Dense(128, activation='relu'))
  model.add(Dense(64, activation='relu'))
  model.add(Dense(1, activation='relu'))
  model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

  return model

#### **Training of the 3-layers Neural Networks**

In [ ]:
model = define_model_3layer()
model.fit(X_train_felm_1, y_train_felm_1, epochs=5, batch_size=32)
loss_felm_1, accuracy_felm_1 = model.evaluate(X_test_felm_1, y_test_felm_1)

Epoch 1/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 5s 51ms/step - accuracy: 0.4874 - loss: 7.5572
Epoch 2/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4689 - loss: 8.4673 
Epoch 3/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4850 - loss: 8.2107 
Epoch 4/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4800 - loss: 8.2893 
Epoch 5/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4597 - loss: 8.6135  
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 115ms/step - accuracy: 0.5332 - loss: 7.4415


In [ ]:
model = define_model_3layer()
model.fit(X_train_felm_4, y_train_felm_4, epochs=5, batch_size=32)
loss_felm_4, accuracy_felm_4 = model.evaluate(X_test_felm_4, y_test_felm_4)

Epoch 1/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 2s 38ms/step - accuracy: 0.4983 - loss: 6.3815
Epoch 2/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5027 - loss: 8.0150 
Epoch 3/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4779 - loss: 8.4153 
Epoch 4/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5268 - loss: 7.6266 
Epoch 5/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5207 - loss: 7.7253 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - accuracy: 0.4668 - loss: 8.5945


In [ ]:
model = define_model_3layer()
model.fit(X_train_felm_8, y_train_felm_8, epochs=5, batch_size=32)
loss_felm_8, accuracy_felm_8 = model.evaluate(X_test_felm_8, y_test_felm_8)

Epoch 1/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 2s 40ms/step - accuracy: 0.4682 - loss: 7.2110
Epoch 2/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5198 - loss: 4.1984 
Epoch 3/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6063 - loss: 1.3678 
Epoch 4/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6911 - loss: 0.8187 
Epoch 5/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7086 - loss: 0.8499 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - accuracy: 0.5828 - loss: 0.6857


In [ ]:
model = define_model_3layer()
model.fit(X_train_felm_16, y_train_felm_16, epochs=5, batch_size=32)
loss_felm_16, accuracy_felm_16 = model.evaluate(X_test_felm_16, y_test_felm_16)

Epoch 1/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 2s 42ms/step - accuracy: 0.5739 - loss: 1.2170
Epoch 2/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6826 - loss: 0.6152 
Epoch 3/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7378 - loss: 0.5250 
Epoch 4/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8435 - loss: 0.3709 
Epoch 5/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8960 - loss: 0.3366 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - accuracy: 0.6084 - loss: 1.5229


In [ ]:
model = define_model_3layer(X_train_felm_all.shape[1])
model.fit(X_train_felm_all, y_train_felm_1, epochs=5, batch_size=32)
loss_felm_all, accuracy_felm_all = model.evaluate(X_test_felm_all, y_test_felm_1)

Epoch 1/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 3s 50ms/step - accuracy: 0.4784 - loss: 7.7023
Epoch 2/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4690 - loss: 8.4652 
Epoch 3/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4728 - loss: 8.4056 
Epoch 4/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4866 - loss: 8.1855 
Epoch 5/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5250 - loss: 7.5720 
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 81ms/step - accuracy: 0.5332 - loss: 7.4415


In [ ]:
model = define_model_3layer()
model.fit(X_train_factalign_1, y_train_factalign_1, epochs=5, batch_size=32)
loss_factalign_1, accuracy_factalign_1 = model.evaluate(X_test_factalign_1, y_test_factalign_1)

Epoch 1/5
65/65 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.5089 - loss: 7.3635
Epoch 2/5
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5219 - loss: 7.6228
Epoch 3/5
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5288 - loss: 7.5113
Epoch 4/5
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5281 - loss: 7.5240
Epoch 5/5
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5144 - loss: 7.7413
17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.4829 - loss: 8.2441


In [ ]:
model = define_model_3layer()
model.fit(X_train_factalign_4, y_train_factalign_4, epochs=5, batch_size=32)
loss_factalign_4, accuracy_factalign_4 = model.evaluate(X_test_factalign_4, y_test_factalign_4)

Epoch 1/5
65/65 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.5034 - loss: 6.4013
Epoch 2/5
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5235 - loss: 7.5964
Epoch 3/5
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5211 - loss: 7.6351
Epoch 4/5
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5386 - loss: 7.3558
Epoch 5/5
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5045 - loss: 7.8990
17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.4829 - loss: 8.2441


In [ ]:
model = define_model_3layer()
model.fit(X_train_factalign_8, y_train_factalign_8, epochs=5, batch_size=32)
loss_factalign_8, accuracy_factalign_8 = model.evaluate(X_test_factalign_8, y_test_factalign_8)

Epoch 1/5
65/65 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - accuracy: 0.4976 - loss: 6.9175
Epoch 2/5
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5139 - loss: 7.7438
Epoch 3/5
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5216 - loss: 7.6273
Epoch 4/5
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5335 - loss: 7.4378
Epoch 5/5
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5392 - loss: 7.3044
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.4829 - loss: 8.2441


In [ ]:
model = define_model_3layer()
model.fit(X_train_factalign_16, y_train_factalign_16, epochs=5, batch_size=32)
loss_factalign_16, accuracy_factalign_16 = model.evaluate(X_test_factalign_16, y_test_factalign_16)

Epoch 1/5
65/65 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.5427 - loss: 3.2642
Epoch 2/5
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6128 - loss: 3.4835
Epoch 3/5
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6606 - loss: 3.3508
Epoch 4/5
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6931 - loss: 3.4869
Epoch 5/5
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6224 - loss: 3.6647
17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.5792 - loss: 4.3602


In [ ]:
model = define_model_3layer(X_train_factalign_all.shape[1])
model.fit(X_train_factalign_all, y_train_factalign_1, epochs=5, batch_size=32)
loss_factalign_all, accuracy_factalign_all = model.evaluate(X_test_factalign_all, y_test_factalign_1)

Epoch 1/5
65/65 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.5309 - loss: 7.3879
Epoch 2/5
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4946 - loss: 8.1460
Epoch 3/5
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4930 - loss: 8.1714
Epoch 4/5
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4819 - loss: 8.3510
Epoch 5/5
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4818 - loss: 8.3528
17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.5171 - loss: 7.7831


In [ ]:
model = define_model_3layer()
model.fit(X_train_factbench_1, y_train_factbench_1, epochs=5, batch_size=32)
loss_factbench_1, accuracy_factbench_1 = model.evaluate(X_test_factbench_1, y_test_factbench_1)

Epoch 1/5
50/50 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step - accuracy: 0.5155 - loss: 7.4393
Epoch 2/5
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4991 - loss: 7.9858
Epoch 3/5
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4921 - loss: 8.0966
Epoch 4/5
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5198 - loss: 7.6562
Epoch 5/5
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4804 - loss: 8.2834
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - accuracy: 0.4881 - loss: 8.1608


In [ ]:
model = define_model_3layer()
model.fit(X_train_factbench_4, y_train_factbench_4, epochs=5, batch_size=32)
loss_factbench_4, accuracy_factbench_4 = model.evaluate(X_test_factbench_4, y_test_factbench_4)

Epoch 1/5
50/50 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.5258 - loss: 7.5708
Epoch 2/5
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5144 - loss: 7.8277
Epoch 3/5
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5114 - loss: 7.8760
Epoch 4/5
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5106 - loss: 7.8880
Epoch 5/5
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5321 - loss: 7.5414
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.5119 - loss: 7.8674


In [ ]:
model = define_model_3layer()
model.fit(X_train_factbench_8, y_train_factbench_8, epochs=5, batch_size=32)
loss_factbench_8, accuracy_factbench_8 = model.evaluate(X_test_factbench_8, y_test_factbench_8)

Epoch 1/5
50/50 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.4958 - loss: 7.2470
Epoch 2/5
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4890 - loss: 8.1462
Epoch 3/5
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4932 - loss: 8.0801
Epoch 4/5
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4969 - loss: 8.0206
Epoch 5/5
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5070 - loss: 7.8594
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.4881 - loss: 8.1608


In [ ]:
model = define_model_3layer()
model.fit(X_train_factbench_16, y_train_factbench_16, epochs=5, batch_size=32)
loss_factbench_16, accuracy_factbench_16 = model.evaluate(X_test_factbench_16, y_test_factbench_16)

Epoch 1/5
50/50 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.5727 - loss: 0.9674
Epoch 2/5
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7984 - loss: 0.5263
Epoch 3/5
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7965 - loss: 0.5728
Epoch 4/5
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7707 - loss: 0.7880
Epoch 5/5
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8737 - loss: 0.4605
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.5807 - loss: 1.5145


In [ ]:
model = define_model_3layer(X_train_factbench_all.shape[1])
model.fit(X_train_factbench_all, y_train_factbench_1, epochs=5, batch_size=32)
loss_factbench_all, accuracy_factbench_all = model.evaluate(X_test_factbench_all, y_test_factbench_1)

Epoch 1/5
50/50 ━━━━━━━━━━━━━━━━━━━━ 3s 20ms/step - accuracy: 0.5135 - loss: 7.4224
Epoch 2/5
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4808 - loss: 8.2771
Epoch 3/5
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4947 - loss: 8.0555
Epoch 4/5
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4760 - loss: 8.3541
Epoch 5/5
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4835 - loss: 8.2340
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - accuracy: 0.4881 - loss: 8.1608


## **Dataframes with metrics**

In [ ]:
metrics_felm_3layer = {
    'classifier' : ['dnn_3_layer', 'dnn_3_layer', 'dnn_3_layer', 'dnn_3_layer', 'dnn_3_layer'],
    'layer' : [-1, -4, -8, -16, 'all'],
    'accuracy' : [accuracy_felm_1, accuracy_felm_4, accuracy_felm_8, accuracy_felm_16, accuracy_felm_all]
}

metrics_factalign_3layer = {
    'classifier' : ['dnn_3_layer', 'dnn_3_layer', 'dnn_3_layer', 'dnn_3_layer', 'dnn_3_layer'],
    'layer' : [-1, -4, -8, -16, 'all'],
    'accuracy' : [accuracy_factalign_1, accuracy_factalign_4, accuracy_factalign_8, accuracy_factalign_16, accuracy_factalign_all]
}

metrics_factbench_3layer = {
    'classifier' : ['dnn_3_layer', 'dnn_3_layer', 'dnn_3_layer', 'dnn_3_layer', 'dnn_3_layer'],
    'layer' : [-1, -4, -8, -16, 'all'],
    'accuracy' : [accuracy_factbench_1, accuracy_factbench_4, accuracy_factbench_8, accuracy_factbench_16, accuracy_factbench_all]
}


In [ ]:
df_metrics_felm_3layer = pd.DataFrame(metrics_felm_3layer)
df_metrics_factalign_3layer = pd.DataFrame(metrics_factalign_3layer)
df_metrics_factbench_3layer = pd.DataFrame(metrics_factbench_3layer)

In [ ]:
df_metrics_felm = pd.concat([df_metrics_felm, df_metrics_felm_3layer])
df_metrics_factalign = pd.concat([df_metrics_factalign, df_metrics_factalign_3layer])
df_metrics_factbench = pd.concat([df_metrics_factbench, df_metrics_factbench_3layer])

In [ ]:
df_metrics_factalign

,classifier,layer,accuracy
0,svm,-1,0.730994
1,svm,-4,0.707602
2,svm,-8,0.709552
3,svm,-16,0.668616
4,svm,all,0.715400
0,lr,-1,0.674464
1,lr,-4,0.680312
2,lr,-8,0.699805
3,lr,-16,0.688109
4,lr,all,0.690058


## **Neural Network - 5 layers**

In [ ]:
## Function that define the deep neural networks with five layers

def define_model_5layer(input_dim = X_train_felm_1.shape[1]):

  model = Sequential()
  model.add(Dense(1024, activation='relu', input_dim=input_dim))
  model.add(Dense(512, activation='relu', input_dim=input_dim))
  model.add(Dense(256, activation='relu', input_dim=input_dim))
  model.add(Dense(128, activation='relu'))
  model.add(Dense(64, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))
  model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

  return model

#### **Training of the 5-layers Neural Networks**

In [ ]:
model = define_model_5layer()
model.fit(X_train_felm_1, y_train_felm_1, epochs=5, batch_size=32)
loss_felm_1_5layer, accuracy_felm_1_5layer = model.evaluate(X_test_felm_1, y_test_felm_1)

model = define_model_5layer()
model.fit(X_train_felm_4, y_train_felm_4, epochs=5, batch_size=32)
loss_felm_4_5layer, accuracy_felm_4_5layer = model.evaluate(X_test_felm_4, y_test_felm_4)

model = define_model_5layer()
model.fit(X_train_felm_8, y_train_felm_8, epochs=5, batch_size=32)
loss_felm_8_5layer, accuracy_felm_8_5layer = model.evaluate(X_test_felm_8, y_test_felm_8)

model = define_model_5layer()
model.fit(X_train_felm_16, y_train_felm_16, epochs=5, batch_size=32)
loss_felm_16_5layer, accuracy_felm_16_5layer = model.evaluate(X_test_felm_16, y_test_felm_16)

model = define_model_5layer(X_train_felm_all.shape[1])
model.fit(X_train_felm_all, y_train_felm_1, epochs=5, batch_size=32)
loss_felm_all_5layer, accuracy_felm_all_5layer = model.evaluate(X_test_felm_all, y_test_felm_1)

model = define_model_5layer()
model.fit(X_train_factalign_1, y_train_factalign_1, epochs=5, batch_size=32)
loss_factalign_1_5layer, accuracy_factalign_1_5layer = model.evaluate(X_test_factalign_1, y_test_factalign_1)

model = define_model_5layer()
model.fit(X_train_factalign_4, y_train_factalign_4, epochs=5, batch_size=32)
loss_factalign_4_5layer, accuracy_factalign_4_5layer = model.evaluate(X_test_factalign_4, y_test_factalign_4)

model = define_model_5layer()
model.fit(X_train_factalign_8, y_train_factalign_8, epochs=5, batch_size=32)
loss_factalign_8_5layer, accuracy_factalign_8_5layer = model.evaluate(X_test_factalign_8, y_test_factalign_8)

model = define_model_5layer()
model.fit(X_train_factalign_16, y_train_factalign_16, epochs=5, batch_size=32)
loss_factalign_16_5layer, accuracy_factalign_16_5layer = model.evaluate(X_test_factalign_16, y_test_factalign_16)

model = define_model_5layer(X_train_factalign_all.shape[1])
model.fit(X_train_factalign_all, y_train_factalign_1, epochs=5, batch_size=32)
loss_factalign_all_5layer, accuracy_factalign_all_5layer = model.evaluate(X_test_factalign_all, y_test_factalign_1)

model = define_model_5layer()
model.fit(X_train_factbench_1, y_train_factbench_1, epochs=5, batch_size=32)
loss_factbench_1_5layer, accuracy_factbench_1_5layer = model.evaluate(X_test_factbench_1, y_test_factbench_1)

model = define_model_5layer()
model.fit(X_train_factbench_4, y_train_factbench_4, epochs=5, batch_size=32)
loss_factbench_4_5layer, accuracy_factbench_4_5layer = model.evaluate(X_test_factbench_4, y_test_factbench_4)

model = define_model_5layer()
model.fit(X_train_factbench_8, y_train_factbench_8, epochs=5, batch_size=32)
loss_factbench_8_5layer, accuracy_factbench_8_5layer = model.evaluate(X_test_factbench_8, y_test_factbench_8)

model = define_model_5layer()
model.fit(X_train_factbench_16, y_train_factbench_16, epochs=5, batch_size=32)
loss_factbench_16_5layer, accuracy_factbench_16_5layer = model.evaluate(X_test_factbench_16, y_test_factbench_16)

model = define_model_5layer(X_train_factbench_all.shape[1])
model.fit(X_train_factbench_all, y_train_factbench_1, epochs=5, batch_size=32)
loss_factbench_all_5layer, accuracy_factbench_all_5layer = model.evaluate(X_test_factbench_all, y_test_factbench_1)

Epoch 1/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 6s 81ms/step - accuracy: 0.4826 - loss: 0.9897
Epoch 2/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6270 - loss: 0.7017 
Epoch 3/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6422 - loss: 0.6617 
Epoch 4/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6482 - loss: 0.5937 
Epoch 5/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5807 - loss: 0.8274 
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 130ms/step - accuracy: 0.5817 - loss: 0.6919
Epoch 1/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 4s 63ms/step - accuracy: 0.5176 - loss: 0.7262
Epoch 2/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5525 - loss: 0.6804 
Epoch 3/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6337 - loss: 0.6631 
Epoch 4/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7704 - loss: 0.5222 
Epoch 5/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7533 - loss: 0.4608 
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 64ms/step - accuracy: 0.5326 - loss: 1.2896
Epoch 1/5
15/15

## **Dataframes with metrics**

In [ ]:
metrics_felm_5layer = {
    'classifier' : ['dnn_5_layer', 'dnn_5_layer', 'dnn_5_layer', 'dnn_5_layer', 'dnn_5_layer'],
    'layer' : [-1, -4, -8, -16, 'all'],
    'accuracy' : [accuracy_felm_1_5layer, accuracy_felm_4_5layer, accuracy_felm_8_5layer, accuracy_felm_16_5layer, accuracy_felm_all_5layer]
}

metrics_factalign_5layer = {
    'classifier' : ['dnn_5_layer', 'dnn_5_layer', 'dnn_5_layer', 'dnn_5_layer', 'dnn_5_layer'],
    'layer' : [-1, -4, -8, -16, 'all'],
    'accuracy' : [accuracy_factalign_1_5layer, accuracy_factalign_4_5layer, accuracy_factalign_8_5layer, accuracy_factalign_16_5layer, accuracy_factalign_all_5layer]
}

metrics_factbench_5layer = {
    'classifier' : ['dnn_5_layer', 'dnn_5_layer', 'dnn_5_layer', 'dnn_5_layer', 'dnn_5_layer'],
    'layer' : [-1, -4, -8, -16, 'all'],
    'accuracy' : [accuracy_factbench_1_5layer, accuracy_factbench_4_5layer, accuracy_factbench_8_5layer, accuracy_factbench_16_5layer, accuracy_factbench_all_5layer]
}


In [ ]:
df_metrics_felm_5layer = pd.DataFrame(metrics_felm_5layer)
df_metrics_factalign_5layer = pd.DataFrame(metrics_factalign_5layer)
df_metrics_factbench_5layer = pd.DataFrame(metrics_factbench_5layer)

In [ ]:
df_metrics_felm = pd.concat([df_metrics_felm, df_metrics_felm_5layer])
df_metrics_factalign = pd.concat([df_metrics_factalign, df_metrics_factalign_5layer])
df_metrics_factbench = pd.concat([df_metrics_factbench, df_metrics_factbench_5layer])

In [ ]:
df_metrics_felm

,classifier,layer,accuracy
0,svm,-1,0.730994
1,svm,-4,0.707602
2,svm,-8,0.709552
3,svm,-16,0.668616
4,svm,all,0.715400
0,lr,-1,0.674464
1,lr,-4,0.680312
2,lr,-8,0.699805
3,lr,-16,0.688109
4,lr,all,0.690058


## **Saving Dataframes**

Saving dataframes with metrics on Google Drive

In [ ]:
saving_path = '/content/drive/MyDrive/Ceccarelli_MasterThesis&Internship/Master Thesis/Results'

In [ ]:
with open(saving_path + "/metrics_saplma_factbench_answer.csv", "w") as f:
    df_metrics_factbench.to_csv(f, index=False)

with open(saving_path + "/metrics_saplma_factalign_answer.csv", "w") as f:
    df_metrics_factalign.to_csv(f, index=False)

with open(saving_path + "/metrics_saplma_felm_answer.csv", "w") as f:
    df_metrics_felm.to_csv(f, index=False)